# 导包

In [2]:
# package imports
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tqdm import tqdm

# 相关全局设置
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_info_rows', 20000000)
sns.set()
np.random.seed(2020)

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() 
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem/(1024*1024)))
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    for col in df.columns:
        col_type = df[col].dtype
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() 
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem/(1024*1024)))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
# train_df = pd.read_csv('./data/input_data/train_data.csv', sep='|')
train_df = pd.read_csv('./data/input_data/train_data.csv', sep='|',skiprows = lambda x: x > 0 and np.random.rand() > 0.1)
test_df = pd.read_csv('./data/input_data/test_data_A.csv',sep='|')


In [5]:
df = pd.concat([train_df,test_df],axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5188108 entries, 0 to 999999
Data columns (total 37 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   label                        4188108 non-null  float64
 1   uid                          5188108 non-null  int64  
 2   task_id                      5188108 non-null  int64  
 3   adv_id                       5188108 non-null  int64  
 4   creat_type_cd                5188108 non-null  int64  
 5   adv_prim_id                  5188108 non-null  int64  
 6   dev_id                       5188108 non-null  int64  
 7   inter_type_cd                5188108 non-null  int64  
 8   slot_id                      5188108 non-null  int64  
 9   spread_app_id                5188108 non-null  int64  
 10  tags                         5188108 non-null  int64  
 11  app_first_class              5188108 non-null  int64  
 12  app_second_class             5188108 non-nu

In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 36 columns):
 #   Column                       Non-Null Count    Dtype 
---  ------                       --------------    ----- 
 0   id                           1000000 non-null  int64 
 1   uid                          1000000 non-null  int64 
 2   task_id                      1000000 non-null  int64 
 3   adv_id                       1000000 non-null  int64 
 4   creat_type_cd                1000000 non-null  int64 
 5   adv_prim_id                  1000000 non-null  int64 
 6   dev_id                       1000000 non-null  int64 
 7   inter_type_cd                1000000 non-null  int64 
 8   slot_id                      1000000 non-null  int64 
 9   spread_app_id                1000000 non-null  int64 
 10  tags                         1000000 non-null  int64 
 11  app_first_class              1000000 non-null  int64 
 12  app_second_class             1000000 non-null  int64 
 13

# 查看缺失值

In [7]:
train_df.head().append(train_df.tail())

#缺失值是-1

,label,uid,task_id,adv_id,creat_type_cd,adv_prim_id,dev_id,inter_type_cd,slot_id,spread_app_id,tags,app_first_class,app_second_class,age,city,city_rank,device_name,device_size,career,gender,net_type,residence,his_app_size,his_on_shelf_time,app_score,emui_dev,list_time,device_price,up_life_duration,up_membership_grade,membership_life_duration,consume_purchase,communication_onlinerate,communication_avgonline_30d,indu_name,pt_d
0,0,1028662,1238,2639,3,142,60,5,12,80,41,2,26,1,311,3,64,156,9,4,3,39,-1,-1,1,14,13,2,-1,-1,-1,2,-1,3,24,1
1,0,1945498,3744,2227,3,142,36,5,12,80,41,4,26,6,331,3,85,193,8,2,3,14,5,3,2,25,17,3,20,1,-1,2,6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23,10,42,1
2,0,1953547,5879,3563,7,150,17,5,11,11,39,4,17,6,179,3,81,162,4,4,2,39,6,3,2,20,4,5,-1,-1,-1,2,7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23,9,36,1
3,0,1247282,1269,5189,7,160,58,5,14,79,41,4,26,5,416,5,32,291,9,2,2,12,9,2,2,20,4,5,20,1,-1,2,2^3^4^5^6^7^8^9^10^11^12^13^14^15^16^17^18^19^...,12,42,1
4,0,1542400,3303,1858,7,206,64,5,13,22,39,4,17,7,103,4,82,117,9,2,2,39,4,3,2,14,9,3,-1,-1,-1,2,5^6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^...,10,36,1
4188103,0,1689438,4147,2693,7,168,37,5,17,44,40,4,25,6,406,5,102,141,3,2,2,21,9,3,2,20,12,4,-1,-1,-1,2,6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23,11,17,7
4188104,0,1031308,5550,3436,7,142,36,4,12,80,41,4,26,4,431,3,86,193,9,2,2,37,5,3,2,25,17,3,12,-1,-1,2,5^6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^...,11,42,7
4188105,0,1573663,5596,4147,7,206,64,5,12,22,39,4,17,5,231,5,46,141,4,4,2,20,4,3,2,14,10,3,18,-1,-1,2,6^7^8^9^10^11^12^13^14^15^16^17^18^19^20^21^22^23,11,36,7
4188106,0,1521168,5382,1250,6,113,60,2,18,78,23,2,23,5,192,3,42,141,4,4,3,40,-1,-1,1,20,8,2,18,-1,-1,2,3^4^5^6^7^8^9^10^11^12^13^14^15^16^17^18^19^20...,11,24,7
4188107,0,2181198,3006,3658,7,156,56,5,11,58,37,4,21,6,402,4,47,297,5,2,2,38,14,3,2,27,17,5,20,1,-1,5,2^3^4^5^6^7^8^9^10^11^12^13^14^15^16^17^18^19^...,12,17,7


# 数据清洗

## 缺失值

In [8]:
df=df.replace([np.inf, -np.inf],0)
df=df.fillna(0)

# 特征工程

## 去掉特征

In [9]:
drop_cols = ['pt_d','label','communication_onlinerate','uid']
df.drop(columns=drop_cols, inplace = True)

## 离散特征

In [10]:
cate_cols = ['city_rank','creat_type_cd','device_size','gender','indu_name','inter_type_cd','residence','slot_id','net_type','task_id','adv_id','adv_prim_id','age','app_first_class','app_second_class','career','city','consume_purchase','dev_id','tags']

for f in tqdm(cate_cols):
    map_dict = dict(zip(df[f].unique(), range(df[f].nunique())))
    df[f] = df[f].map(map_dict).fillna(-1).astype('int32')
    df[f + '_count'] = df[f].map(df[f].value_counts())

100%|██████████| 20/20 [00:12<00:00,  1.63it/s]


## 连续特征

In [11]:
sparse_features = cate_cols
dense_features = [x for x in df.columns if x not in cate_cols] #这里的dense_feature可以把树模型的特征加进来
print('sparse_feature: {}'.format(sparse_features))
print('dense_feature: {}'.format(dense_features))

mms = MinMaxScaler(feature_range=(0,1))
df[dense_features] = mms.fit_transform(df[dense_features])

sparse_feature: ['city_rank', 'creat_type_cd', 'device_size', 'gender', 'indu_name', 'inter_type_cd', 'residence', 'slot_id', 'net_type', 'task_id', 'adv_id', 'adv_prim_id', 'age', 'app_first_class', 'app_second_class', 'career', 'city', 'consume_purchase', 'dev_id', 'tags']
dense_feature: ['spread_app_id', 'device_name', 'his_app_size', 'his_on_shelf_time', 'app_score', 'emui_dev', 'list_time', 'device_price', 'up_life_duration', 'up_membership_grade', 'membership_life_duration', 'communication_avgonline_30d', 'id', 'city_rank_count', 'creat_type_cd_count', 'device_size_count', 'gender_count', 'indu_name_count', 'inter_type_cd_count', 'residence_count', 'slot_id_count', 'net_type_count', 'task_id_count', 'adv_id_count', 'adv_prim_id_count', 'age_count', 'app_first_class_count', 'app_second_class_count', 'career_count', 'city_count', 'consume_purchase_count', 'dev_id_count', 'tags_count']


In [12]:
df = reduce_mem_usage(df)

Memory usage of dataframe is 1741.61 MB
Memory usage after optimization is: 484.88 MB
Decreased by 72.2%


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5188108 entries, 0 to 999999
Data columns (total 53 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   task_id                      5188108 non-null  int16  
 1   adv_id                       5188108 non-null  int16  
 2   creat_type_cd                5188108 non-null  int8   
 3   adv_prim_id                  5188108 non-null  int8   
 4   dev_id                       5188108 non-null  int8   
 5   inter_type_cd                5188108 non-null  int8   
 6   slot_id                      5188108 non-null  int8   
 7   spread_app_id                5188108 non-null  float16
 8   tags                         5188108 non-null  int8   
 9   app_first_class              5188108 non-null  int8   
 10  app_second_class             5188108 non-null  int8   
 11  age                          5188108 non-null  int8   
 12  city                         5188108 non-nu

In [21]:
df.head().append(df.tail())

,task_id,adv_id,creat_type_cd,adv_prim_id,dev_id,inter_type_cd,slot_id,spread_app_id,tags,app_first_class,app_second_class,age,city,city_rank,device_name,device_size,career,gender,net_type,residence,his_app_size,his_on_shelf_time,app_score,emui_dev,list_time,device_price,up_life_duration,up_membership_grade,membership_life_duration,consume_purchase,communication_avgonline_30d,indu_name,id,city_rank_count,creat_type_cd_count,device_size_count,gender_count,indu_name_count,inter_type_cd_count,residence_count,slot_id_count,net_type_count,task_id_count,adv_id_count,adv_prim_id_count,age_count,app_first_class_count,app_second_class_count,career_count,city_count,consume_purchase_count,dev_id_count,tags_count
0,0,0,0,0,0,0,0,0.884766,0,0,0,0,0,0,0.569824,0,0,0,0,0,0.000000,0.00,0.0,0.176514,0.789551,0.375,0.000000,0.0,0.0,0,0.285645,0,0.0,1.000000,0.100647,0.101379,0.278076,0.205811,1.000000,0.239258,1.000000,0.253906,0.000117,0.000117,0.856934,0.065430,0.280518,0.400146,0.961914,0.042542,1.000000,1.000000,0.408936
1,1,1,0,0,1,0,0,0.884766,0,1,0,1,1,0,0.795898,1,1,1,0,1,0.272705,1.00,1.0,0.823730,1.000000,0.500,1.000000,0.5,0.0,0,0.785645,1,0.0,1.000000,0.100647,0.131714,1.000000,0.213013,1.000000,0.284668,1.000000,0.253906,0.314941,0.314941,0.856934,0.850586,1.000000,0.400146,0.036530,0.045685,1.000000,0.349854,0.408936
2,2,2,1,1,2,0,1,0.000000,1,1,1,1,2,0,0.752930,2,2,0,1,0,0.318115,1.00,1.0,0.529297,0.315674,0.750,0.000000,0.0,0.0,0,0.714355,2,0.0,1.000000,1.000000,0.154907,0.278076,0.316650,1.000000,0.239258,0.453857,1.000000,0.039856,0.039856,0.279541,0.850586,1.000000,0.490967,1.000000,0.039398,1.000000,0.765137,0.472412
3,3,3,1,2,3,0,2,0.871582,0,1,0,2,3,1,0.225830,3,0,1,1,2,0.454590,0.75,1.0,0.529297,0.315674,0.750,1.000000,0.5,0.0,0,0.928711,1,0.0,0.742188,1.000000,0.007187,1.000000,0.213013,1.000000,0.497559,0.268311,1.000000,0.018463,0.009193,0.013382,1.000000,1.000000,0.400146,0.961914,0.329346,1.000000,0.006283,0.408936
4,4,4,1,3,4,0,3,0.140991,1,1,1,3,4,2,0.763672,4,0,1,1,0,0.227295,1.00,1.0,0.176514,0.579102,0.500,0.000000,0.0,0.0,0,0.785645,2,0.0,0.389404,1.000000,0.188110,1.000000,0.316650,1.000000,0.239258,0.329590,1.000000,0.043091,0.043091,0.190552,0.513672,1.000000,0.490967,0.961914,0.136719,1.000000,0.088257,0.472412
999995,4024,4883,2,23,0,1,4,0.858887,4,0,4,2,176,0,0.290283,2,2,1,0,23,0.000000,0.00,0.0,0.529297,0.315674,0.625,0.904785,0.0,0.0,1,0.928711,0,1.0,1.000000,0.214966,0.154907,1.000000,0.205811,0.152954,0.103638,0.911133,0.253906,0.009651,0.009651,0.139282,1.000000,0.280518,0.322754,1.000000,0.098450,0.063538,1.000000,0.345215
999996,3621,4328,2,22,0,1,1,0.858887,4,0,4,1,304,0,0.483887,5,2,1,0,21,0.000000,0.00,0.0,0.529297,0.526367,0.625,1.000000,0.5,0.0,0,0.785645,10,1.0,1.000000,0.214966,1.000000,1.000000,0.014366,0.152954,0.220337,0.453857,0.253906,0.308594,0.071350,0.062225,0.850586,0.280518,0.322754,1.000000,0.033081,1.000000,1.000000,0.345215
999997,4008,4867,3,0,0,0,0,0.884766,0,0,0,2,173,0,0.817383,4,3,1,0,22,0.000000,0.00,0.0,1.000000,0.368408,0.625,1.000000,0.5,0.0,0,0.856934,1,1.0,1.000000,0.056641,0.188110,1.000000,0.213013,1.000000,0.160522,1.000000,0.253906,0.014336,0.014336,0.856934,1.000000,0.280518,0.400146,0.058533,0.077881,1.000000,1.000000,0.408936
999998,447,451,1,11,7,0,4,0.500000,2,1,2,1,166,1,0.290283,2,0,1,1,2,0.318115,1.00,1.0,0.529297,0.315674,0.625,1.000000,0.0,0.0,0,0.785645,3,1.0,0.742188,1.000000,0.154907,1.000000,1.000000,1.000000,0.497559,0.911133,1.000000,0.066223,0.066223,0.189941,0.850586,1.000000,1.000000,0.961914,0.154907,1.000000,0.679199,1.000000
999999,4109,4970,1,103,0,0,4,0.756348,1,0,1,2,72,0,0.483887,5,4,1,1,7,0.000000,0.00,0.0,0.529297,0.526367,0.625,0.000000,0.0,0.0,0,0.856934,0,1.0,1.000000,1.000000,1.000000,1.000000,0.205811,1.000000,1.000000,0.911133,1.000000,0.010468,0.010468,0.009880,1.000000,0.280518,0.490967,0.052704,0.034149,1.000000,1.000000,0.472412


# 保存数据

In [14]:
df.to_csv('./data/all_df.csv', index=False)

In [15]:
train_ready_df = df[:-1000000]
train_ready_df['label'] = train_df['label']

test_ready_df = df[-1000000:]

In [16]:
train_ready_df.info()
test_ready_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4188108 entries, 0 to 4188107
Data columns (total 54 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   task_id                      4188108 non-null  int16  
 1   adv_id                       4188108 non-null  int16  
 2   creat_type_cd                4188108 non-null  int8   
 3   adv_prim_id                  4188108 non-null  int8   
 4   dev_id                       4188108 non-null  int8   
 5   inter_type_cd                4188108 non-null  int8   
 6   slot_id                      4188108 non-null  int8   
 7   spread_app_id                4188108 non-null  float16
 8   tags                         4188108 non-null  int8   
 9   app_first_class              4188108 non-null  int8   
 10  app_second_class             4188108 non-null  int8   
 11  age                          4188108 non-null  int8   
 12  city                         4188108 non-n

In [17]:
train_ready_df.to_csv('./data/train_df.csv', index=False)
test_ready_df.to_csv('./data/test_df.csv', index=False)